In [12]:
!ls

Data_Concatination.ipynb Suumo.Rmd                suumo_kawasaki.csv
Scraping.ipynb           suumo_adachi.csv


In [30]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time

#URL（東京都足立区の賃貸住宅情報 検索結果の1ページ目）
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13109&ta=13&cb=0.0&ct=9999999&md=01&md=02&md=03&md=04&et=9999999&mb=0&mt=9999999&cn=9999999&fw2='

#データ取得
result = requests.get(url)
c = result.content

#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c)

#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})

#ページ数を取得
body = soup.find("body")
pages = body.find_all("div",{'class':'pagination pagination_set-nav'})
pages_text = str(pages)
pages_split = pages_text.split('</a></li>\n</ol>')
pages_split0 = pages_split[0]
pages_split1 = pages_split0[-3:]
pages_split2 = pages_split1.replace('>','')
pages_split3 = int(pages_split2)

#URLを入れるリスト
urls = []

#1ページ目を格納
urls.append(url)

#2ページ目から最後のページまでを格納
for i in range(pages_split3-1):
    pg = str(i+2)
    url_page = url + '&pn=' + pg
    urls.append(url_page)
    
# name = [] #マンション名
# address = [] #住所
# locations0 = [] #立地1つ目（最寄駅/徒歩~分）
# locations1 = [] #立地2つ目（最寄駅/徒歩~分）
# locations2 = [] #立地3つ目（最寄駅/徒歩~分）
# age = [] #築年数
# height = [] #建物高さ
# floor = [] #階
# rent = [] #賃料
# admin = [] #管理費
# others = [] #敷/礼/保証/敷引,償却
# floor_plan = [] #間取り
# area = [] #専有面積

#各ページで以下の動作をループ
for url in urls:
    
    name = [] #マンション名
    address = [] #住所
    locations0 = [] #立地1つ目（最寄駅/徒歩~分）
    locations1 = [] #立地2つ目（最寄駅/徒歩~分）
    locations2 = [] #立地3つ目（最寄駅/徒歩~分）
    age = [] #築年数
    height = [] #建物高さ
    floor = [] #階
    rent = [] #賃料
    admin = [] #管理費
    others = [] #敷/礼/保証/敷引,償却
    floor_plan = [] #間取り
    area = [] #専有面積
    
    #物件リストを切り出し
    result = requests.get(url)
    print('acessing page no '+str(url.split('&pn=')[-1]))
    c = result.content
    soup = BeautifulSoup(c)
    summary = soup.find("div",{'id':'js-bukkenList'})
    
    #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
    try:
        cassetteitems = summary.find_all("div",{'class':'cassetteitem'})
        print('no of house in a page: '+str(len(cassetteitems)))
    except:
        print('cannot read something...')
        continue
    #各cassetteitemsに対し、以下の動作をループ
    for i in range(len(cassetteitems)):
        #各建物から売りに出ている部屋数を取得
        tbodies = cassetteitems[i].find_all('tbody')
        
        #マンション名取得
        subtitle = cassetteitems[i].find_all("div",{
            'class':'cassetteitem_content-title'})
        subtitle = str(subtitle)
        subtitle_rep = subtitle.replace(
            '[<div class="cassetteitem_content-title">', '')
        subtitle_rep2 = subtitle_rep.replace(
            '</div>]', '')

        #住所取得
        subaddress = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col1'})
        subaddress = str(subaddress)
        subaddress_rep = subaddress.replace(
            '[<li class="cassetteitem_detail-col1">', '')
        subaddress_rep2 = subaddress_rep.replace(
            '</li>]', '')
        
        #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
        for y in range(len(tbodies)):
            name.append(subtitle_rep2)
            address.append(subaddress_rep2)

        #立地を取得
        sublocations = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col2'})
        
        #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
        for x in sublocations:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        locations0.append(text)
                    elif i == 1:
                        locations1.append(text)
                    elif i == 2:
                        locations2.append(text)
                        
        #築年数と建物高さを取得
        tbodies = cassetteitems[i].find_all('tbody')
        col3 = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col3'})
        for x in col3:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        age.append(text)
                    else:
                        height.append(text)

    #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
    tables = summary.find_all('table')

    #各建物（table）に対して、売りに出ている部屋（row）を取得
    rows = []
    for i in range(len(tables)):
        rows.append(tables[i].find_all('tr'))

    #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
    data = []
    for row in rows:
        for tr in row:
            cols = tr.find_all('td')
            for td in cols:
                text = td.find(text=True)
                data.append(text)

    #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
    index = 0
    for item in data:
        if '階' in item:
            floor.append(data[index])
            rent.append(data[index+1])
            admin.append(data[index+2])
            others.append(data[index+3])
            floor_plan.append(data[index+4])
            area.append(data[index+5])
        index +=1
    
    #プログラムを10秒間停止する（スクレイピングマナー）
    time.sleep(10)

#各リストをシリーズ化
    name = Series(name)
    address = Series(address)
    locations0 = Series(locations0)
    locations1 = Series(locations1)
    locations2 = Series(locations2)
    age = Series(age)
    height = Series(height)
    floor = Series(floor)
    rent = Series(rent)
    admin = Series(admin)
    others = Series(others)
    floor_plan = Series(floor_plan)
    area = Series(area)

#各シリーズをデータフレーム化
    suumo_df = pd.concat([name, address, locations0, locations1,
                      locations2, age, height,floor,rent,admin,others,floor_plan,area],axis=1)

#カラム名
# suumo_df.columns=['マンション名','住所','立地1','立地2','立地3','築年数','建物高さ','階','賃料','管理費',
#                   '敷/礼/保証/敷引,償却','間取り','専有面積']

#csvファイルとして保存

    suumo_df.to_csv('suumo_shinagawa.csv', mode='a',sep = '\t',encoding='utf-16', header=False)

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


acessing page no https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13109&ta=13&cb=0.0&ct=9999999&md=01&md=02&md=03&md=04&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=
no of house in a page: 30
acessing page no 2
no of house in a page: 30
acessing page no 3
no of house in a page: 30
acessing page no 4
no of house in a page: 30
acessing page no 5
no of house in a page: 30
acessing page no 6
no of house in a page: 30
acessing page no 7
no of house in a page: 30
acessing page no 8
no of house in a page: 30
acessing page no 9
no of house in a page: 30
acessing page no 10
no of house in a page: 30
acessing page no 11
no of house in a page: 30
acessing page no 12
no of house in a page: 30
acessing page no 13
no of house in a page: 30
acessing page no 14
no of house in a page: 30
acessing page no 15
no of house in a page: 30
acessing page no 16
no of house in a page: 30
acessing page no 17
no of house in a page: 30
acessi

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time

#URL（東京都足立区の賃貸住宅情報 検索結果の1ページ目）
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=14&sa=02&cb=0.0&ct=9999999&mb=0&mt=9999999&md=01&md=02&md=03&md=04&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=12&pn=4'

#データ取得
result = requests.get(url)
c = result.content

#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c)

#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})


#URLを入れるリスト
urls = []

#1ページ目を格納
urls.append(url)

    
# name = [] #マンション名
# address = [] #住所
# locations0 = [] #立地1つ目（最寄駅/徒歩~分）
# locations1 = [] #立地2つ目（最寄駅/徒歩~分）
# locations2 = [] #立地3つ目（最寄駅/徒歩~分）
# age = [] #築年数
# height = [] #建物高さ
# floor = [] #階
# rent = [] #賃料
# admin = [] #管理費
# others = [] #敷/礼/保証/敷引,償却
# floor_plan = [] #間取り
# area = [] #専有面積

#各ページで以下の動作をループ
for url in urls:
    
    name = [] #マンション名
    address = [] #住所
    locations0 = [] #立地1つ目（最寄駅/徒歩~分）
    locations1 = [] #立地2つ目（最寄駅/徒歩~分）
    locations2 = [] #立地3つ目（最寄駅/徒歩~分）
    age = [] #築年数
    height = [] #建物高さ
    floor = [] #階
    rent = [] #賃料
    admin = [] #管理費
    others = [] #敷/礼/保証/敷引,償却
    floor_plan = [] #間取り
    area = [] #専有面積
    
    #物件リストを切り出し
    result = requests.get(url)
    print('acessing page no '+str(url.split('&pn=')[-1]))
    c = result.content
    soup = BeautifulSoup(c)
    summary = soup.find("div",{'id':'js-bukkenList'})
    
    #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
    try:
        cassetteitems = summary.find_all("div",{'class':'cassetteitem'})
        print('no of house in a page: '+str(len(cassetteitems)))
    except:
        print('cannot read something...')
        continue
    #各cassetteitemsに対し、以下の動作をループ
    for i in range(len(cassetteitems)):
        #各建物から売りに出ている部屋数を取得
        tbodies = cassetteitems[i].find_all('tbody')
        
        #マンション名取得
        subtitle = cassetteitems[i].find_all("div",{
            'class':'cassetteitem_content-title'})
        subtitle = str(subtitle)
        subtitle_rep = subtitle.replace(
            '[<div class="cassetteitem_content-title">', '')
        subtitle_rep2 = subtitle_rep.replace(
            '</div>]', '')

        #住所取得
        subaddress = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col1'})
        subaddress = str(subaddress)
        subaddress_rep = subaddress.replace(
            '[<li class="cassetteitem_detail-col1">', '')
        subaddress_rep2 = subaddress_rep.replace(
            '</li>]', '')
        
        #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
        for y in range(len(tbodies)):
            name.append(subtitle_rep2)
            address.append(subaddress_rep2)

        #立地を取得
        sublocations = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col2'})
        
        #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
        for x in sublocations:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        locations0.append(text)
                    elif i == 1:
                        locations1.append(text)
                    elif i == 2:
                        locations2.append(text)

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


acessing page no 4
no of house in a page: 30


In [9]:
locations2

['小田急線/向ヶ丘遊園駅 歩33分',
 'ＪＲ南武線/稲田堤駅 歩26分',
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/百合ヶ丘駅 歩24分',
 'ＪＲ京浜東北線/川崎駅 バス12分 (バス停)小田栄 歩3分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 '小田急線/新百合ヶ丘駅 歩20分',
 None,
 None,
 'ＪＲ南武線/武蔵中原駅 歩16分',
 None,
 'ＪＲ南武線/稲田堤駅 歩24分',
 '小田急線/新百合ヶ丘駅 バス17分 (バス停)東長沢 歩6分',
 'ＪＲ南武線/中野島駅 歩38分',
 'ＪＲ南武線/中野島駅 歩38分',
 None,
 None,
 None,
 'ＪＲ南武線/尻手駅 歩18分',
 '小田急線/柿生駅 歩35分',
 None,
 'ＪＲ南武線/登戸駅 歩33分',
 '小田急線/読売ランド前駅 歩30分',
 '小田急線/読売ランド前駅 歩30分',
 '小田急線/読売ランド前駅 歩30分',
 '小田急線/登戸駅 歩27分',
 '小田急線/登戸駅 歩27分',
 '小田急線/登戸駅 歩27分',
 'ＪＲ南武線/尻手駅 歩19分',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/百合ヶ丘駅 歩24分',
 '小田急線/向ヶ丘遊園駅 バス11分 (バス停)生田駅前北口 歩4分',
 '京急大師線/東門前駅 歩16分',
 '小田急線/生田駅 歩20分',
 '小田急線/生田駅 歩20分',
 '小田急線/向ヶ丘遊園駅 歩26分',
 'ＪＲ南武線/